In [2]:
%matplotlib inline


from msmbuilder.featurizer import DihedralFeaturizer
from msmbuilder.decomposition import PCA
from msmbuilder.preprocessing import RobustScaler
#from msmbuilder.cluster import MiniBatchKMeans
from msmbuilder.cluster import KCenters
from msmbuilder.msm import MarkovStateModel
from msmbuilder.io.sampling import sample_dimension
from msmbuilder.io import load_trajs, save_generic, preload_top, backup

import numpy as np

import msmexplorer as msme

rs = np.random.RandomState(42)

/Users/sbamos/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/sbamos/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/sbamos/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core imp

In [3]:
from msmbuilder.dataset import dataset
import os
import mdtraj as mdt
import glob


cwd = os.getcwd()
print(cwd)

## Load
trajs = dataset("*SC*.xtc", topology="md.loop_SC_start.gro")
    
for traj in trajs:
    print(traj)

/Users/sbamos/Documents/WORK/MSM
<mdtraj.Trajectory with 5001 frames, 447 atoms, 41 residues, and unitcells>
<mdtraj.Trajectory with 5001 frames, 447 atoms, 41 residues, and unitcells>
<mdtraj.Trajectory with 5001 frames, 447 atoms, 41 residues, and unitcells>
<mdtraj.Trajectory with 5001 frames, 447 atoms, 41 residues, and unitcells>
<mdtraj.Trajectory with 5001 frames, 447 atoms, 41 residues, and unitcells>


In [ ]:
featurizer = DihedralFeaturizer(types=['phi', 'psi'])
diheds = featurizer.fit_transform(trajs)

print(np.shape(trajs))

In [ ]:
scaler = RobustScaler()
scaled_data = scaler.fit_transform(diheds)

In [ ]:
pca_model = PCA(n_components=2)
pca_trajs = pca_model.fit_transform(scaled_data)

ax, side_ax = msme.plot_trace(pca_trajs[0][:, 0], window=10,
                              label='PC1', xlabel='Timestep')
_ = msme.plot_trace(pca_trajs[0][:, 1], window=10, label='PC2',
                    xlabel='Timestep', color='rawdenim', ax=ax,
                    side_ax=side_ax)

In [ ]:
pca_model = PCA(n_components=5)
pca_trajs = pca_model.fit_transform(scaled_data)

ax, side_ax = msme.plot_trace(pca_trajs[0][:, 0], window=10,
                              label='PC1', xlabel='Timestep')
_ = msme.plot_trace(pca_trajs[0][:, 1], window=10, label='PC2',
                    xlabel='Timestep', color='rawdenim', ax=ax,
                    side_ax=side_ax)
__ = msme.plot_trace(pca_trajs[0][:, 2], window=10, label='PC3',
                    xlabel='Timestep', color='dijon', ax=ax,
                    side_ax=side_ax)
___ = msme.plot_trace(pca_trajs[0][:, 3], window=10, label='PC4',
                    xlabel='Timestep', color='carbon', ax=ax,
                    side_ax=side_ax)
____ = msme.plot_trace(pca_trajs[0][:, 4], window=10, label='PC5',
                    xlabel='Timestep', color='pomegranate', ax=ax,
                    side_ax=side_ax)

In [ ]:
%matplotlib inline
import msmexplorer as msme
import numpy as np
txx = np.concatenate(pca_trajs)
_ = msme.plot_histogram(txx, labels=['$PC1$', '$PC2$'],
                    show_titles=True)

In [ ]:
clusterer = KCenters(n_clusters=12, random_state=rs)
clustered_trajs = clusterer.fit_transform(pca_trajs)

_ = msme.plot_voronoi(clusterer, xlabel='PC1', ylabel='PC2')

In [ ]:
msm = MarkovStateModel(lag_time=100, n_timescales=5)
assigns = msm.fit_transform(clustered_trajs)

_ = msme.plot_pop_resids(msm, color='tarragon')

In [ ]:
_ = msme.plot_timescales(msm, ylabel=r'Relaxation Time (frames)')

In [ ]:
msm_list = [
    MarkovStateModel(lag_time=x, n_timescales=5, verbose=False)
                     for x in [1, 5, 25, 125]
]

for msm in msm_list:
    msm.fit(clustered_trajs)

In [ ]:
_ = msme.plot_implied_timescales(msm_list,
                                  xlabel=r'$\tau$ (frames)',
                                  ylabel='Relaxation Times (frames)')

In [ ]:
msm = msm_list[2]  # Choose the appropriate MSM from the list

msm = MarkovStateModel(lag_time=40, n_timescales=5)
assigns = msm.fit_transform(clustered_trajs)

In [ ]:
for i, (ts, ts_u) in enumerate(zip(msm.timescales_, msm.uncertainty_timescales())):
    timescale_ns = ts * 50 / 1000
    uncertainty_ns = ts_u * 50 / 1000
    print('Timescale %d: %.2f ± %.2f ns' % ((i + 1), timescale_ns, uncertainty_ns))

In [ ]:
data = np.concatenate(pca_trajs, axis=0)
pi_0 = msm.populations_[np.concatenate(assigns, axis=0)]


# Free Energy Surface
ax = msme.plot_free_energy(data, obs=(0, 1), n_samples=10000,
                          pi=pi_0, gridsize=100, vmax=5.,
                          n_levels=8, cut=5, xlabel='PC1',
                          ylabel='PC2', random_state=rs)

# MSM Network
pos = dict(zip(range(clusterer.n_clusters), clusterer.cluster_centers_))
_ = msme.plot_msm_network(msm, pos=pos, node_color='carbon',
                          with_labels=False)


# Top Transition Pathway
w = (msm.left_eigenvectors_[:, 1] - msm.left_eigenvectors_[:, 1].min())
w /= w.max()
cmap = msme.utils.make_colormap(['rawdenim', 'lightgrey', 'pomegranate'])
_ = msme.plot_tpaths(msm, [1], [0], pos=pos, node_color=cmap(w),
                     alpha=.9, edge_color='black', ax=ax)

Save coordinates

In [ ]:
from msmbuilder.io import gather_metadata, save_meta, NumberedRunsParser

## Construct and save the dataframe
parser = NumberedRunsParser(
    traj_fmt="md.loop_SC_{run}.xtc",
    top_fn="md.loop_SC_start.gro",
    step_ps=100,
)
meta = gather_metadata("*SC*.xtc", parser)
save_meta(meta)

print(meta)
print(trajs)


In [ ]:
meta, ttrajs = load_trajs('trajs', meta=meta)